# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/new-york-city-taxi-fare-prediction/sample_submission.csv
/kaggle/input/new-york-city-taxi-fare-prediction/GCP-Coupons-Instructions.rtf
/kaggle/input/new-york-city-taxi-fare-prediction/train.csv
/kaggle/input/new-york-city-taxi-fare-prediction/test.csv


მონაცემების ამორება და დაბეჭდვა

In [2]:

train_df = pd.read_csv('/kaggle/input/new-york-city-taxi-fare-prediction/train.csv', nrows=1000)
test_df = pd.read_csv('/kaggle/input/new-york-city-taxi-fare-prediction/test.csv')

print(train_df.describe())


       fare_amount  pickup_longitude  pickup_latitude  dropoff_longitude  \
count  1000.000000       1000.000000      1000.000000        1000.000000   
mean     11.469940        -72.126972        39.732521         -72.198714   
std       9.499792         11.555398         6.365536          11.327382   
min       2.500000        -74.438233         0.000000         -74.185996   
25%       6.000000        -73.993044        40.735310         -73.991830   
50%       8.500000        -73.982135        40.752876         -73.980246   
75%      13.000000        -73.966590        40.766689         -73.962534   
max      66.300000          0.000000        40.993260           0.000000   

       dropoff_latitude  passenger_count  
count       1000.000000      1000.000000  
mean          39.773237         1.592000  
std            6.240137         1.222715  
min            0.000000         0.000000  
25%           40.732978         1.000000  
50%           40.753924         1.000000  
75%           

გამოტოვებული მნიშვნელობების ამოშლა

In [3]:
train_df.dropna(inplace=True)

მონაცემების შეზღუდვა (ამგვარად შედეგი უფრო ზუსტი იქნება), ვზღუდავთ ნიუ იორკის ფარგლებში (ნიუ იორკის გრძედებსა და განედებს შორის), რადგან გამოჩნდა რომ მინიმალური fare amount და passenger count იყო 0.

In [4]:
train_df = train_df[(train_df['fare_amount'] > 0) & (train_df['fare_amount'] < 500)]
train_df = train_df[(train_df['passenger_count'] > 0) & (train_df['passenger_count'] <= 6)]
train_df = train_df[(train_df['pickup_longitude'] > -80) & (train_df['pickup_longitude'] < -70)]
train_df = train_df[(train_df['pickup_latitude'] > 35) & (train_df['pickup_latitude'] < 45)]
train_df = train_df[(train_df['dropoff_longitude'] > -80) & (train_df['dropoff_longitude'] < -70)]
train_df = train_df[(train_df['dropoff_latitude'] > 35) & (train_df['dropoff_latitude'] < 45)]

feature engineering - > ვხდით მოცემულ ინფორმაციას უფრო რელევანტურს გამოყენებისთვის. ამჯერად Haversine formulaს გამოყენებით ვანგარიშობთ სფეროზე(დედამიწაზე) დისტანციას ორ წერტილს შორის

In [5]:
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    a = np.sin(delta_phi / 2) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2) ** 2
    return R * (2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)))

train_df['haversine_distance'] = haversine_distance(train_df['pickup_latitude'], train_df['pickup_longitude'],
                                                    train_df['dropoff_latitude'], train_df['dropoff_longitude'])


ევკლიდეს დისტანცია

In [6]:
def euclidean_distance(lat1, lon1, lat2, lon2):
    return np.sqrt((lat1 - lat2)**2 + (lon1 - lon2)**2)

train_df['euclidean_distance'] = euclidean_distance(train_df['pickup_latitude'], train_df['pickup_longitude'],
                                                    train_df['dropoff_latitude'], train_df['dropoff_longitude'])


აეროპორტის კოორდინატები

In [7]:
jfk_coords = (40.6413, -73.7781)
lga_coords = (40.7769, -73.8740)
ewr_coords = (40.6895, -74.1745)

მანძილები აეროპორტამდე

In [8]:
train_df['pickup_distance_to_jfk'] = haversine_distance(train_df['pickup_latitude'], train_df['pickup_longitude'], jfk_coords[0], jfk_coords[1])
train_df['dropoff_distance_to_jfk'] = haversine_distance(train_df['dropoff_latitude'], train_df['dropoff_longitude'], jfk_coords[0], jfk_coords[1])
train_df['pickup_distance_to_lga'] = haversine_distance(train_df['pickup_latitude'], train_df['pickup_longitude'], lga_coords[0], lga_coords[1])
train_df['dropoff_distance_to_lga'] = haversine_distance(train_df['dropoff_latitude'], train_df['dropoff_longitude'], lga_coords[0], lga_coords[1])
train_df['pickup_distance_to_ewr'] = haversine_distance(train_df['pickup_latitude'], train_df['pickup_longitude'], ewr_coords[0], ewr_coords[1])
train_df['dropoff_distance_to_ewr'] = haversine_distance(train_df['dropoff_latitude'], train_df['dropoff_longitude'], ewr_coords[0], ewr_coords[1])


პოპულარულ ადგილებამდე მანძილები

In [9]:
times_square_coords = (40.7580, -73.9855)
central_park_coords = (40.7851, -73.9683)
wall_street_coords = (40.7074, -74.0113)

train_df['pickup_distance_to_times_square'] = haversine_distance(train_df['pickup_latitude'], train_df['pickup_longitude'], times_square_coords[0], times_square_coords[1])
train_df['dropoff_distance_to_times_square'] = haversine_distance(train_df['dropoff_latitude'], train_df['dropoff_longitude'], times_square_coords[0], times_square_coords[1])
train_df['pickup_distance_to_central_park'] = haversine_distance(train_df['pickup_latitude'], train_df['pickup_longitude'], central_park_coords[0], central_park_coords[1])
train_df['dropoff_distance_to_central_park'] = haversine_distance(train_df['dropoff_latitude'], train_df['dropoff_longitude'], central_park_coords[0], central_park_coords[1])
train_df['pickup_distance_to_wall_street'] = haversine_distance(train_df['pickup_latitude'], train_df['pickup_longitude'], wall_street_coords[0], wall_street_coords[1])
train_df['dropoff_distance_to_wall_street'] = haversine_distance(train_df['dropoff_latitude'], train_df['dropoff_longitude'], wall_street_coords[0], wall_street_coords[1])


დროის მონაცემები

In [10]:
train_df['pickup_datetime'] = pd.to_datetime(train_df['pickup_datetime'])
train_df['hour'] = train_df['pickup_datetime'].dt.hour
train_df['day_of_week'] = train_df['pickup_datetime'].dt.dayofweek
train_df['month'] = train_df['pickup_datetime'].dt.month

დისტანცია ნიუ იორკის ცენტრამდე

In [11]:

nyc_center = (-74.0060, 40.7128)

def distance_to_center(lat, lon, center_lat, center_lon):
    return haversine_distance(lat, lon, center_lat, center_lon)

train_df['pickup_distance_to_center'] = distance_to_center(train_df['pickup_latitude'], train_df['pickup_longitude'], nyc_center[1], nyc_center[0])
train_df['dropoff_distance_to_center'] = distance_to_center(train_df['dropoff_latitude'], train_df['dropoff_longitude'], nyc_center[1], nyc_center[0])



In [12]:
features = ['haversine_distance', 'euclidean_distance', 'hour', 'day_of_week', 'month', 'passenger_count', 
            'pickup_distance_to_center', 'dropoff_distance_to_center',
            'pickup_distance_to_jfk', 'dropoff_distance_to_jfk', 
            'pickup_distance_to_lga', 'dropoff_distance_to_lga',
            'pickup_distance_to_ewr', 'dropoff_distance_to_ewr']
X = train_df[features]
y = train_df['fare_amount']


მონაცემების დაყოფა

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


მნიშვნელობების მინიჭება

In [14]:
test_df['haversine_distance'] = haversine_distance(test_df['pickup_latitude'], test_df['pickup_longitude'],
                                                   test_df['dropoff_latitude'], test_df['dropoff_longitude'])
test_df['euclidean_distance'] = euclidean_distance(test_df['pickup_latitude'], test_df['pickup_longitude'],
                                                   test_df['dropoff_latitude'], test_df['dropoff_longitude'])
test_df['pickup_distance_to_jfk'] = haversine_distance(test_df['pickup_latitude'], test_df['pickup_longitude'], jfk_coords[0], jfk_coords[1])
test_df['dropoff_distance_to_jfk'] = haversine_distance(test_df['dropoff_latitude'], test_df['dropoff_longitude'], jfk_coords[0], jfk_coords[1])
test_df['pickup_distance_to_lga'] = haversine_distance(test_df['pickup_latitude'], test_df['pickup_longitude'], lga_coords[0], lga_coords[1])
test_df['dropoff_distance_to_lga'] = haversine_distance(test_df['dropoff_latitude'], test_df['dropoff_longitude'], lga_coords[0], lga_coords[1])
test_df['pickup_distance_to_ewr'] = haversine_distance(test_df['pickup_latitude'], test_df['pickup_longitude'], ewr_coords[0], ewr_coords[1])
test_df['dropoff_distance_to_ewr'] = haversine_distance(test_df['dropoff_latitude'], test_df['dropoff_longitude'], ewr_coords[0], ewr_coords[1])
test_df['pickup_distance_to_times_square'] = haversine_distance(test_df['pickup_latitude'], test_df['pickup_longitude'], times_square_coords[0], times_square_coords[1])
test_df['dropoff_distance_to_times_square'] = haversine_distance(test_df['dropoff_latitude'], test_df['dropoff_longitude'], times_square_coords[0], times_square_coords[1])
test_df['pickup_distance_to_central_park'] = haversine_distance(test_df['pickup_latitude'], test_df['pickup_longitude'], central_park_coords[0], central_park_coords[1])
test_df['dropoff_distance_to_central_park'] = haversine_distance(test_df['dropoff_latitude'], test_df['dropoff_longitude'], central_park_coords[0], central_park_coords[1])
test_df['pickup_distance_to_wall_street'] = haversine_distance(test_df['pickup_latitude'], test_df['pickup_longitude'], wall_street_coords[0], wall_street_coords[1])
test_df['dropoff_distance_to_wall_street'] = haversine_distance(test_df['dropoff_latitude'], test_df['dropoff_longitude'], wall_street_coords[0], wall_street_coords[1])
test_df['pickup_datetime'] = pd.to_datetime(test_df['pickup_datetime'])
test_df['hour'] = test_df['pickup_datetime'].dt.hour
test_df['day_of_week'] = test_df['pickup_datetime'].dt.dayofweek
test_df['month'] = test_df['pickup_datetime'].dt.month
test_df['pickup_distance_to_center'] = distance_to_center(test_df['pickup_latitude'], test_df['pickup_longitude'], nyc_center[1], nyc_center[0])
test_df['dropoff_distance_to_center'] = distance_to_center(test_df['dropoff_latitude'], test_df['dropoff_longitude'], nyc_center[1], nyc_center[0])
X_test = test_df[features]

წრფივი რეგრესია

In [15]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_val)
rmse_lr = np.sqrt(mean_squared_error(y_val, y_pred_lr))
print(f'Linear Regression Validation RMSE: {rmse_lr:.2f}')

test_df['fare_amount_lr'] = lr.predict(X_test)


Linear Regression Validation RMSE: 5.41


In [16]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

sample_size = min(10000, len(train_df))

small_train_df = train_df.sample(n=sample_size, random_state=42)

X_small = small_train_df[features]
y_small = small_train_df['fare_amount']

rf = RandomForestRegressor(random_state=42, n_jobs=-1)

param_distributions = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 15, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2']
}

random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_distributions, 
                                   n_iter=10, cv=3, n_jobs=-1, random_state=42, verbose=0)

random_search.fit(X_small, y_small)

best_rf = random_search.best_estimator_

best_rf.fit(X_train, y_train)
y_pred_rf = best_rf.predict(X_val)
rmse_rf = np.sqrt(mean_squared_error(y_val, y_pred_rf))
print(f'Random Forest Validation RMSE: {rmse_rf:.2f}')

# Predictions on test set
test_df['fare_amount_rf'] = best_rf.predict(X_test)


Random Forest Validation RMSE: 4.54


შედეგის გამოტანა

In [17]:
print(f'Linear Regression RMSE: {rmse_lr:.2f}')
print(f'Random Forest RMSE: {rmse_rf:.2f}')

submission_lr = test_df[['key', 'fare_amount_lr']].rename(columns={'fare_amount_lr': 'fare_amount'})
submission_rf = test_df[['key', 'fare_amount_rf']].rename(columns={'fare_amount_rf': 'fare_amount'})

submission_lr.to_csv('submission_lr.csv', index=False)
submission_rf.to_csv('submission_rf.csv', index=False)

Linear Regression RMSE: 5.41
Random Forest RMSE: 4.54
